# Permutation generation
In this notebook, we show how the 30 3-qubit operators from our paper were constructed in a way to ensure that all 3-qubit permutations are Clifford equivalent with these 30.

## Imports and setup

In [ ]:
import numpy as np
import itertools
from qiskit.quantum_info import Pauli
from tqdm import tqdm

In [ ]:
n_qubits = 3
output_folder = "data/input/64/permutations/"

## Generating the Puali matrices

In [ ]:
def generate_paulis():
    """
    Generate all possible Pauli matrices for a given number of qubits.
    
    Returns:
        numpy.ndarray: Array of Pauli matrices.
    """
    matrices = []
    for combination in itertools.product(["I", "X", "Y", "Z"], repeat=n_qubits):
        matrices.append(Pauli("".join(combination)).to_matrix())
    
    return np.array(matrices)

paulis = generate_paulis()

## Defining useful functions

In [ ]:
def check_clifford(permutation_matrix, paulis, epsilon=1e-8):
    """
    Check if a permutation matrix satisfies the Clifford equivalence condition for a set of Pauli matrices.

    Parameters:
        permutation_matrix (numpy.ndarray): The permutation matrix to be checked.
        paulis (list[numpy.ndarray]): The list of Pauli matrices.
        epsilon (float, optional): The tolerance value for numerical comparisons. Defaults to 1e-8.

    Returns:
        bool: True if the permutation matrix satisfies the Clifford equivalence condition, False otherwise.
    """
    computed_all = np.conjugate(np.matmul(np.matmul(permutation_matrix, paulis), permutation_matrix.T))
    for i in range(len(paulis)):
        norms = np.array([np.abs(np.trace(np.matmul(pauli, computed_all[i]))) for pauli in paulis])
        if np.all(norms < 2 ** n_qubits - epsilon):
            return False
        
    return True

def do_permutation(permutation):
    """
    Apply a permutation to the identity matrix and return the resulting matrix.

    Args:
        permutation (list): A list representing the permutation.

    Returns:
        numpy.ndarray: The matrix obtained by applying the permutation to the identity matrix.
    """
    matrix = np.identity(len(permutation))
    perm_matrix = [[] for i in range(len(permutation))]
    for i, perm in enumerate(permutation):
        perm_matrix[i] = matrix[perm]

    return np.array(perm_matrix)

### Generating the operators by looping over all possible permutations

Note that this will take approximately one hour, since there are 40000 possible permutations.

In [ ]:
identity = np.identity(2 ** n_qubits)
unique_matrices = []
pbar = tqdm(itertools.permutations(identity))
for i, permutation in enumerate(pbar):
    to_add = True
    for other_matrix in unique_matrices:
        if check_clifford(np.matmul(np.array(permutation), other_matrix.T), paulis):
            to_add = False
            break
    
    if to_add:
        unique_matrices.append(np.array(permutation))
        pbar.set_description_str(str(len(unique_matrices)))

### Add the operators to the output folder

In [ ]:
for i, matrix in  enumerate(unique_matrices):
    with open(output_folder + f"{i}.txt", "w") as f:
        f.write("matrix\n")
        f.write(f"{n_qubits}\n")

        for line in matrix:
            f.write(" ".join([f"({n},0)" for n in line.astype(np.int32)]))
            f.write("\n")
        
        for j in range(len(matrix)):
            f.write(" ".join(["1" for _ in range(len(matrix))]))
            if j != len(matrix) - 1:
                f.write("\n")